# Standard Setup For Kaggle Inputs

In [ ]:
from pathlib import Path

BASE_INPUT = Path("/kaggle/input")
OUTPUT_DIR = Path("/kaggle/working")

print("content of /kaggle/input:")
for item in BASE_INPUT.iterdir():
    tipo = "DIR " if item.is_dir() else "FILE"
    print(f" - [{tipo}] {item.name}")

#Search for "data.yaml"
yaml_path = None
for path in BASE_INPUT.rglob("data.yaml"):
    yaml_path = path
    break

if yaml_path is None:
    print("\n ERROR:: no 'data.yaml' found in /kaggle/input.")
    print("Check if your Roboflow dataset was correctly uploaded.")
else:
     
    DATA_DIR = yaml_path.parent

    print(f"\n 'data.yaml' found: {yaml_path}")
    print("DATA_DIR:", DATA_DIR)
    print("OUTPUT_DIR           :", OUTPUT_DIR)

    print("\n DATA_DIR content :")
    for item in DATA_DIR.iterdir():
        tipo = "DIR " if item.is_dir() else "FILE"
        print(f" - [{tipo}] {item.name}")



# Imports and Dataset Splits

In [ ]:
from pathlib import Path
import yaml
import torch

#DATA_DIR check
if "DATA_DIR" not in globals():
    raise RuntimeError("DATA_DIR is not defined. Execute the first cell.")

data_dir = Path(DATA_DIR)
yaml_file = data_dir / "data.yaml"

if not yaml_file.exists():
    yaml_file = None
    for path in data_dir.rglob("data.yaml"):
        yaml_file = path
        break

if yaml_file is None or not yaml_file.exists():
    print("Critical Error: 'data.yaml' not found in DATA_DIR.")
else:
    print(f"Using configuration file: {yaml_file}")

    with open(yaml_file, "r") as f:
        data_cfg = yaml.safe_load(f)

    print("\n Principal Content of data.yaml:")
    for k in ["path", "train", "val", "test", "nc", "names"]:
        if k in data_cfg:
            print(f" - {k}: {data_cfg[k]}")

    base_path = yaml_file.parent  #/kaggle/input/food-dataset

    def find_images_dir(split_name, yaml_key):
        """
        Finds the folder <split>/images looking in this order:
        1) path said in data.yaml
        2) base_path/<split_name>/images
        3) recursive scan in base_path
        """
        # 1) path said in data.yaml
        p_str = data_cfg.get(yaml_key)
        if p_str:
            p = Path(p_str)
            if not p.is_absolute():
                p = (base_path / p).resolve()
            if p.exists():
                return p

        # 2) base_path/<split_name>/images
        candidate = base_path / split_name / "images"
        if candidate.exists():
            return candidate

        # 3) recursive scan in base_path
        for img_dir in base_path.rglob("images"):
            if img_dir.parent.name == split_name:
                return img_dir

        return None

    #Images folder
    train_images_dir = find_images_dir("train", "train")
    val_images_dir   = find_images_dir("valid", "val")  
    test_images_dir  = find_images_dir("test", "test")  

    print("\n Images folder:")
    print(f"   train: {train_images_dir}")
    print(f"   val  : {val_images_dir}")
    print(f"   test : {test_images_dir}")

    # Labels folders
    def labels_dir_from_images_dir(img_dir):
        if img_dir and img_dir.exists():
            labels_dir = img_dir.parent / "labels"
            return labels_dir if labels_dir.exists() else None
        return None

    train_labels_dir = labels_dir_from_images_dir(train_images_dir)
    val_labels_dir   = labels_dir_from_images_dir(val_images_dir)
    test_labels_dir  = labels_dir_from_images_dir(test_images_dir)

    print("\n Labels found:")
    print(f"   train: {train_labels_dir}")
    print(f"   val  : {val_labels_dir}")
    print(f"   test : {test_labels_dir}")

    #Image count for split
    def count_images(folder):
        if folder and folder.exists():
            exts = {".jpg", ".jpeg", ".png", ".bmp"}
            return sum(1 for p in folder.rglob("*") if p.suffix.lower() in exts)
        return 0

    print("\n Number of images in splits:")
    print(f"   train: {count_images(train_images_dir)}")
    print(f"   val  : {count_images(val_images_dir)}")
    print(f"   test : {count_images(test_images_dir)}")

    #GPU Check
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\n Current device: {device}")
    if device.type == "cuda":
        print("GPU is ready")
    else:
        print("You're using the CPU. Go in Kaggle 'Settings' -> 'Accelerator' and select a GPU.")


# Ultralytics Install

In [ ]:
#We install ultralytics ignoring the depencies that we noticed were in conflict
!pip install ultralytics==8.4.7 "numpy<2.1.0" --no-deps
#Separate install for other YOLO dependencies
!pip install ultralytics-thop pyyaml --quiet

# Model Choice

In [ ]:
###Test
import sys
import numpy as np
import ultralytics
from ultralytics import YOLO

print(f"Versione NumPy: {np.__version__}")
print(f"Versione Ultralytics: {ultralytics.__version__}")

try:
    # test model loading
    model = YOLO("yolo26s.pt") 
    print(" Success the model is yolo26")
except Exception as e:
    print(f"Critical Error: {e}")

# YAML Creation

In [ ]:
from pathlib import Path
import yaml

#Looks for the dataset
dataset_root = "/kaggle/input/food-dataset"


orig_yaml = Path(f"{dataset_root}/data.yaml")
with open(orig_yaml, "r") as f:
    data_cfg = yaml.safe_load(f)

names = data_cfg["names"]
nc = len(names)

new_cfg = {
    "path": dataset_root,  
    "train": "train/images",
    "val": "valid/images",
    "test": "test/images",
    "nc": nc,
    "names": names,
}


out_yaml = Path("/kaggle/working/allergen30.yaml")
with open(out_yaml, "w") as f:
    yaml.safe_dump(new_cfg, f, sort_keys=False)

print(f"Created YAML for YOLO26: {out_yaml}")
print(f"Classe 0: {names[0]} | Total classes: {nc}")


GPU check

In [ ]:
!nvidia-smi

# Model training with AdamW optimizer

In [ ]:
import os
from ultralytics import YOLO, settings

#Settings reset to avoid conflicts with Ray/WandB/MLFlow
settings.reset()
settings.update({
    "raytune": False, 
    "wandb": False, 
    "mlflow": False, 
    "neptune": False
})

#Loadig pre trained YOLO26
model = YOLO("yolo26s.pt")

#No AMP to avoid crashing
results = model.train(
    data="/kaggle/working/allergen30.yaml",
    epochs=100,
    patience=25,
    imgsz=416,
    batch=32,
    workers=2,
    amp=False,          
    
#AdamW Hyperparameters     
    optimizer='AdamW',    
    lr0=0.001,             
    cos_lr=True,           
    label_smoothing=0.1,   
    mosaic=1.0,            
    mixup=0.15,
    weight_decay=0.0005, 
    #
    
    device=0,
    name="yolo26s_allergeni_TUNED",
    project="allergen_project",
    plots=True
)

# Creation of Confusion Matrix and various plots

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os

#results path
results_dir = "/kaggle/working/runs/detect/allergen_project/yolo26s_allergeni_TUNED2"

#Graphs (Loss, Precision, Recall, mAP)
results_png = os.path.join(results_dir, "results.png")
if os.path.exists(results_png):
    img = cv2.imread(results_png)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(20, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title("Training: Loss and Metrics (P, R, mAP)", fontsize=20)
    plt.show()

#Confusion Matrix
conf_matrix = os.path.join(results_dir, "confusion_matrix.png")
if os.path.exists(conf_matrix):
    img_cm = cv2.imread(conf_matrix)
    img_cm = cv2.cvtColor(img_cm, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(15, 15))
    plt.imshow(img_cm)
    plt.axis('off')
    plt.title("Confusion Matrix", fontsize=20)
    plt.show()

# Validation

In [ ]:
#Loading best weights
model_trained = YOLO(f"{results_dir}/weights/best.pt")

#Validation on test set
metrics = model_trained.val(split='test')

print("\n--- Final Performance on Test Set ---")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

# Export of the best weights in ONNX

In [ ]:
#Loads the trained model
model = YOLO("/kaggle/working/runs/detect/allergen_project/yolo26s_allergeni_TUNED2/weights/best.pt")

#ONNX Export for the app
path = model.export(format="onnx", imgsz=640, opset=17, simplify=True)

print(f"ONNX model reday for download: {path}")

# Check of uploaded models

In [ ]:
import glob
pt_files = glob.glob("/kaggle/input/**/*.pt", recursive=True)

print("Found weights files:")
for file in pt_files:
    print(file)

# Moves uploaded model in kaggle working area then patches it and exports it for the app

In [ ]:

from ultralytics import YOLO
import torch
import copy
import shutil
import os

source_path = "/kaggle/input/bestyolos/pytorch/default/1/best.pt"
writable_path = "best.pt"

print(f"Copying from {source_path} to {writable_path}...")
shutil.copy(source_path, writable_path)


print("--- STARTING YOLO26 COMPATIBILITY EXPORT ---")

model = YOLO("best.pt")
internal_model = model.model

#Patching the ARGS
print("Patching internal configuration...")
if hasattr(internal_model, 'args'):
    args = internal_model.args
    if isinstance(args, dict):
        args['end2end'] = False
        args['nms'] = False
        print("settings patched: end2end=False")
    else:
        args.end2end = False
        args.nms = False
        print(" settings patched: end2end=False")

# Check if the patch worked
for m in internal_model.modules():
    if hasattr(m, 'end2end'):
        print(f"Detect Head status: end2end={m.end2end}") 
        #force-override the class property below 
        if m.end2end == True:
            print(" Property is locked. Applying Nuclear Patch...")
            type(m).end2end = property(lambda self: False) # Force it to return False
            print(f"Nuclear Patch applied. New status: {m.end2end}")

#Prepare for export
internal_model.eval()
dummy_input = torch.randn(1, 3, 640, 640)

#Export directly with PYtorch
print("Exporting via torch.onnx (Bypassing YOLO Exporter)...")

torch.onnx.export(
    internal_model,
    dummy_input,
    "final_model.onnx",
    opset_version=12,
    input_names=['images'],
    output_names=['output0'],
    dynamic_axes=None 
)

print("--- SUCCESS ---")
print("Download 'final_model.onnx'. This is the one.")

# Final plots and Comparison of our models

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from math import pi

data = {
    'Model': ['Nano', 'Small', 'Medium'],
    
    'mAP50': [0.7301, 0.7691, 0.7465], 
    
    'mAP50-95': [0.5347, 0.5726, 0.5740],
    
    'Precision': [0.8226, 0.8352, 0.8376],
    
    'Recall': [0.6362, 0.6636, 0.6405]
}

df = pd.DataFrame(data)

#f1 computing
df['F1_Score'] = 2 * (df['Precision'] * df['Recall']) / (df['Precision'] + df['Recall'])

print("--- MODEL PERFORMANCE TABLE ---")
display(df.round(4))

#colmuns plots

df_melted = df.melt(id_vars="Model", var_name="Metric", value_name="Score")

plt.figure(figsize=(14, 6))
sns.set_style("whitegrid")

chart = sns.barplot(x="Metric", y="Score", hue="Model", data=df_melted, palette="viridis")

for container in chart.containers:
    chart.bar_label(container, fmt='%.2f', padding=3)

plt.title("Model Comparison: Nano vs Small vs Medium", fontsize=16, weight='bold')
plt.ylim(0, 1.0) # Fix y-axis from 0 to 1
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

#radar plot

labels = list(df.columns[1:]) 
num_vars = len(labels)

angles = [n / float(num_vars) * 2 * pi for n in range(num_vars)]
angles += angles[:1] 

plt.figure(figsize=(8, 8))
ax = plt.subplot(111, polar=True)

plt.xticks(angles[:-1], labels)

ax.set_rlabel_position(0)
plt.yticks([0.2, 0.4, 0.6, 0.8], ["0.2", "0.4", "0.6", "0.8"], color="grey", size=7)
plt.ylim(0, 1)

colors = ['#1f77b4', '#ff7f0e', '#2ca02c'] 
for i, row in df.iterrows():
    values = row[1:].tolist()
    values += values[:1] 
    ax.plot(angles, values, linewidth=2, linestyle='solid', label=row['Model'], color=colors[i])
    ax.fill(angles, values, color=colors[i], alpha=0.1)

plt.title("Model Trade-off Analysis (Radar Chart)", size=15, weight='bold', y=1.1)
plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
plt.show()